In [31]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("used_car_price_predictor")

2023/08/18 21:12:53 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/08/18 21:12:53 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/Users/oshan/GoogleDrives/EEDrive/OshanProjects/20230513_zoomcamp_mlops/homework/Project/mlruns/1', creation_time=1692373373730, experiment_id='1', last_update_time=1692373373730, lifecycle_stage='active', name='used_car_price_predictor', tags={}>

In [37]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [34]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [2]:
use_sample= True
random_state = 1024

In [3]:
path_data = Path('data/vehicles.feather')
if not path_data.exists():
    df_cars = pd.read_csv("data/vehicles.csv")
#     save as .feather for future. Faster.
    df_cars.to_feather(path_data)
else :
    df_cars = pd.read_feather(path_data)

Check for missing values

In [4]:
df_cars.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
VIN             161042
drive           130567
size            306361
type             92858
paint_color     130203
image_url           68
description         70
county          426880
state                0
lat               6549
long              6549
posting_date        68
dtype: int64

Since there are a lot of missing values, we will use the columns with few missing values

In [5]:
cols = ['id','year','fuel','odometer','transmission', 'price']
df_cars = df_cars[cols]

In [6]:
df_cars = df_cars.loc[~df_cars.isna().any(axis = 1)]

In [7]:
if use_sample:
    df = df_cars.sample(frac = .1, random_state=random_state)
else:
    df = df_cars.copy()

Ensure that the ids are unique in the dataset.

If not, select first instance of each id.

In [8]:
try:
    assert len(df.id.unique()) == df.shape[0]
except:
    print("there are multiple rows for the same ID")
    print("selecting the first row for each ID")
    df.groupby(['id']).first().reset_index(drop =False,inplace=True)

convert all string dtypes to categorical

In [9]:
# from pandas.api.types import is_string_dtype
# for col in df.columns:
#     if is_string_dtype(df[col]):
#         df[col] = pd.Categorical(df[col])
# df.dtypes

In [10]:
categorical = ['fuel','transmission']
numeric = ['year','odometer']
target = 'price'

In [11]:
df[categorical] = df[categorical].astype(str)

Split to train, validation and test datasets.

- Train dataset       - old cars (cars with year < 2017) => 74%
- Validation dataset  - newer cars (2017 <= year <=2018) => 15%
- Test dataset        - newest cars (2019 <= year)       => 11%

In [12]:
df.head()

,id,year,fuel,odometer,transmission,price
411308,7311450486,2013.0,gas,46267.0,automatic,15995
340463,7303076328,2020.0,gas,21938.0,other,25990
311669,7314370012,1998.0,gas,320220.0,automatic,123
262500,7306571989,2017.0,diesel,29014.0,other,19590
33923,7316374412,2005.0,gas,130000.0,automatic,1500


In [13]:
(pd.DataFrame(df.year.value_counts(normalize=True))
 .reset_index(drop = False)
 .sort_values(by = 'year', ascending = False)
 .head(10))

,year,proportion
59,2022.0,0.000431
22,2021.0,0.005724
9,2020.0,0.044857
6,2019.0,0.060615
0,2018.0,0.088205
1,2017.0,0.083990
3,2016.0,0.071776
2,2015.0,0.074267
5,2014.0,0.069333
4,2013.0,0.071081


In [14]:
dftr = df.loc[df.year < 2017].copy()
dfval = df.loc[df.year.isin([2017, 2018])].copy()
dfte = df.loc[df.year > 2018].copy()

Fit DictVectorizer for categorical columns

In [17]:
dv = DictVectorizer()
tr_dicts = dftr[numeric + categorical].to_dict(orient = 'records')
xtr = dv.fit_transform(tr_dicts)
ytr = dftr[target].values
val_dicts = dfval[numeric + categorical].to_dict(orient='records')
xval = dv.transform(val_dicts)
yval = dfval[target].values
te_dicts = dfte[numeric+categorical].to_dict(orient = 'records')
xte = dv.transform(te_dicts)
yte = dfte[target].values

Train Models

In [18]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [19]:
train = xgb.DMatrix(xtr, label=ytr)
valid = xgb.DMatrix(xval, label=yval)

In [21]:
import numpy as np

In [38]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        ypred = booster.predict(valid)
        rmse = mean_squared_error(yval, ypred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[21:17:13] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19885.54804                                                                                                                       
[1]	validation-rmse:20024.27587                                                                                                                       
[2]	validation-rmse:20147.89309                                                                                                                       
[3]	validation-rmse:20146.83311                                                                                                                       
[4]	validation-rmse:20219.31990                                                                                                                       
[5]	validation-rmse:20257.36528

In [41]:
best_result

{'learning_rate': 0.06464387855883201,
 'max_depth': 5.0,
 'min_child_weight': 0.7542199125563908,
 'reg_alpha': 0.08996209349530984,
 'reg_lambda': 0.1880912290247961}